In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import collections
import subprocess
import glob
from math import floor, ceil
from sklearn.utils import shuffle

import tensorflow as tf
import matplotlib.pyplot as plt 
from tensorflow.keras.layers import ConvLSTM2D, Dense, Flatten, Dropout, Input, BatchNormalization
from tensorflow.keras.models import Model
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [2]:

IMAGE = (64,64)
T = 10
def video_to_frames(path_to_vid):
   
    cap = cv2.VideoCapture(path_to_vid)
    n_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    frm_list = []
    counter = 0
    valid_from = ceil(0.15*n_frames)
    valid_to = floor(0.1*n_frames)
    num_frames = n_frames-valid_from-valid_to
    frame_to_count = floor(num_frames/T)
    no_of_frames_extracted = 0
    while no_of_frames_extracted < T:
        ret, frame = cap.read()
        if ret==True:
            if counter>=valid_from and (counter-valid_from)//frame_to_count==0:
                img = cv2.resize(frame,IMAGE)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                np.expand_dims(gray,axis=1)
                frm_list.append(gray)
                no_of_frames_extracted+=1
                counter+=1
            else:
                counter+=1
        else:
            break
            
    return frm_list

In [3]:
frm_list = video_to_frames('/home/tarun/Kinetic600/kinetics-downloader/dataset/train/juggling_balls/_9ZQNnjgdME.mp4')

In [4]:
print(len(frm_list))

10


In [5]:
def data_generator(input_dir,input_class):
    X = []
    Y = []
    no_of_videos_skipped = 0
    videos = [vid for vid in os.listdir(input_dir) if vid.endswith('.mp4')]
    print("Starting at : ",str(len(videos)))
    for vid in videos:
        frame_list = video_to_frames(os.path.join(input_dir,vid))
        # print(frame_list)
        if len(frame_list)  <  T:
            no_of_videos_skipped+=1
            print('Removed')
            videos.remove(vid)

        else:
            X.append(frame_list)
            y = [0]*2  
            y[input_class] = 1  #one hot encoading
            Y.append(y)
            
    X = np.asarray(X)
    Y = np.asarray(Y)
    return np.expand_dims(X,-1),Y

In [6]:
X1,Y1 = data_generator('/home/tarun/Kinetic600/kinetics-downloader/dataset/train/juggling_balls',0)
X2,Y2 = data_generator('/home/tarun/Kinetic600/kinetics-downloader/dataset/train/punching_bag',1)

X_train = np.concatenate((X1, X2))
y_train = np.concatenate((Y1, Y2))

X_train,y_train = shuffle(X_train,y_train)

Starting at :  694
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Starting at :  845
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed
Removed


In [7]:
print(X_train.shape)
print(y_train.shape)
np.save('tarin_array',X_train)

(1334, 10, 64, 64, 1)
(1334, 2)


In [8]:
X_test1,y_test1 = data_generator('/home/tarun/Kinetic600/kinetics-downloader/dataset/valid/juggling_balls',0)
X_test2,y_test2 = data_generator('/home/tarun/Kinetic600/kinetics-downloader/dataset/valid/punching_bag',1)

X_test = np.concatenate((X_test1, X_test2))
y_test = np.concatenate((y_test1, y_test2))

X_test,y_test = shuffle(X_test,y_test)

Starting at :  40
Removed
Starting at :  41
Removed
Removed
Removed
Removed
Removed


In [10]:
T = 10
IMAGE = (64,64)
i = Input(shape=(T, 64, 64, 1))
x = ConvLSTM2D(filters=64, kernel_size=(3,3), padding='same',return_sequences = True)(i)
# x = BatchNormalization()(x)
x = Dropout(0.2)(x)
# x = ConvLSTM2D(filters=64, kernel_size=(3,3), padding='same',return_sequences = False)(x)
# x = Dropout(0.2)(x)
x = Flatten()(x)
x = Dense(64, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(2,activation='softmax')(x)

model = Model(i,x)
model.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9),loss='binary_crossentropy',metrics=['accuracy'])

model.summary()

r = model.fit(X_test,y_test,batch_size=2,epochs=10)
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save('test.h5')

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 10, 64, 64, 1)]   0         
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 10, 64, 64, 64)    150016    
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 64, 64, 64)    0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2621440)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                167772224 
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130 

ResourceExhaustedError:  OOM when allocating tensor with shape[2621440,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradient_tape/model_1/dense_2/MatMul_1 (defined at <ipython-input-10-fd41aba6c67f>:19) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_7937]

Function call stack:
train_function
